# Plant Classification Using Pytorch
##### Name: Agosh Saini
##### Website: agoshsaini.com

In this section we install kaggle and give kaggle the right permissions

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

This is where we install torch and torchvision. Torch is the module used for machine learning and torchvision is geared towards image based applications. 

In [ ]:
! pip install torch
! pip install torchvision

We want to download the dataset and unzip folder. The rm -rf command is there because we want to remove the images we don't end up using. The kaggle api .json file need to be present in the directory

In [ ]:
! kaggle datasets download msheriey/104-flowers-garden-of-eden 

In [ ]:
! unzip 104-flowers-garden-of-eden

In [ ]:
! rm -rf jpeg-192x192/
! rm -rf jpeg-224x224/
! rm -rf jpeg-311x311/
! rm 104-flowers-garden-of-eden.zip

Setting up the imports in this section and setting up path information. Using a gpu can speed up the training of the data a lot. We need to set up pytorch so it uses the gpu if it is available. 

In [ ]:
'''
Name: Agosh Saini
Contact: Agosh.Saini@gmail.com
Website: agoshsaini.com
'''


# --- Imports --- #
import torch
import torchvision

# --- path to dataset --- #
path = 'jpeg-512x512'

# --- making it work with gpu --- #
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
    
print("Using - " + str(device_name))

Using cuda.


Tranforming images is an important part of the the image classificaiton processes. It is a way to get the model to classify "curveball" images.

In [ ]:
# --- randomly transform images --- #
train_data_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomRotation(180),
    torchvision.transforms.RandomResizedCrop(300),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.45, 0.45, 0.45], 
                                     std=[0.2, 0.2, 0.2])
    ])

test_data_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomRotation(180),
    torchvision.transforms.RandomResizedCrop(300),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.45, 0.45, 0.45], 
                                     std=[0.2, 0.2, 0.2])
    ])

Dataloaders are a great tool for managing the traing of data. They ensure all the data is interated properly while traing and validating out model. Pytorch needs the model, data, and labels being on the same device, so we are sending the data and the labels to gpu if available. Batch size controls how much data is sent through the model at a time controls the rate at which an epoch finishes. However, it can lead to a bad model if made too large.

In [ ]:
# --- creating splits for training and testing --- #
train_data = torchvision.datasets.ImageFolder(path + '/train', 
                                              transform=train_data_transform)
test_data = torchvision.datasets.ImageFolder(path + '/val', 
                                             transform=test_data_transform)
# --- dataloader information --- #
batch_size = 16

train_loader = torch.utils.data.DataLoader(train_data, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, 
                                          batch_size=batch_size, 
                                          shuffle=True)

# --- send the data and labels to gpu if available --- #
for inputs, labels in train_loader:
    inputs, labels = inputs.to(device_name), labels.to(device_name)
for inputs, labels in test_loader:
    inputs, labels = inputs.to(device_name), labels.to(device_name)

We are using the resnet 18 model. It has 18 layers to it and the resnet model allows for skipping of blocks. The loss fucntion and the optimizer are defined here as well. We are sending the model to the gpu if avaiable as well. Learning rate is related to the step size and the momentum is a way to overcome local minimas.

In [ ]:
# --- model parameters --- #
model = torchvision.models.resnet18(pretrained=True)
features = model.fc.in_features
model.fc = torch.nn.Linear(features, len(train_data.classes))

# --- loss function and optimizer --- #
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# --- send model to gpu if available --- #
model.to(device_name)

This is where we train the model. We are trying to minimize the loss function in the model. We are training for 5 passes of the dataset here which is what an epoch refers to. 

In [ ]:
# --- training the last layer --- #
epochs = 5

for epoch in range(epochs):
    running_loss = 0

    for i, (inputs, labels) in enumerate(train_loader):

        optimizer.zero_grad()
        outputs = model(inputs.cuda())
        loss = loss_func(outputs, labels.cuda())
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.cuda().size(0)

    epoch_loss = running_loss / len(train_data)
    print('Epoch ' + str(epoch + 1) + ' of ' + str(epochs) + ' Loss: ' + str(epoch_loss))


# --- saving the model --- #
torch.save(model.state_dict(), 'flower_classifier.pth')

We are testing the model here using the var data in the subfloder. 

In [ ]:
# --- testing the model --- #
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.cuda())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.cuda().size(0)
        correct += (predicted == labels.cuda()).sum().item()

accuracy = correct / total

print("Accuracy: " + str(accuracy))

Accuracy: 0.7567349137931034
